In [1]:
import pandas as pd
from pyalex import Works, Authors, Sources, Institutions, Concepts, Publishers, Funders
from tqdm import tqdm
from multiprocess import Process, Manager
from typing import List, Dict
import json
from joblib import delayed, Parallel
import itertools
from itertools import chain

In [2]:
df = pd.read_csv("fapesp_subset_ciencias_da_computacao.csv", delimiter=";", index_col=False)

/tmp/ipykernel_8606/3672204618.py:1: ParserWarning: Length of header or names does not match length of data. This leads to a loss of data with index_col=False.
  df = pd.read_csv("fapesp_subset_ciencias_da_computacao.csv", delimiter=";", index_col=False)


In [3]:
df #.head(5)

,N. Processo,Título (Português),Título (Inglês),Beneficiário,Instituição,Cidade Instituição,Instituição Parceira,Empresa,Município,Pesquisador Responsável,...,Assuntos,Data de Início,Data de Término,Acordo(s)/Convênio(s) de Cooperação com a FAPESP,Instituições no Exterior,País (Inst. Ext. / Convênio),Pesquisador responsável no exterior,Resumo (Português),Resumo (Inglês),Processos Vinculados
0,23/04895-4,O número de Ramsey relativo a arestas e grafos...,Size-Ramsey numbers and Ramsey-finiteness of g...,Meysam Miralaei,Universidade de São Paulo (USP). Instituto de ...,São Paulo,NaN,NaN,NaN,Yoshiharu Kohayakawa,...,Combinatória:Teoria de Ramsey:Grafos:Métodos p...,2024-05-01,2026-04-30,NaN,NaN,NaN,NaN,Este projeto tem foco em problemas sobre o núm...,This project focuses on numerical problems inv...,NaN
1,23/17002-8,Verificação de identidade de Usuários em redes...,User Identity Verification in CBDC Networks Us...,João Pedro Alonso Almeida,Universidade de São Paulo (USP). Instituto de ...,São Carlos,NaN,NaN,NaN,Jó Ueyama,...,Blockchain,2024-03-25,2024-07-24,NaN,Queensland University Of Technology,Austrália,Raja Jurdak,Uma Moeda Digital de Banco Central (CBDC) repr...,A Central Bank Digital Currency (CBDC) represe...,NaN
2,23/13688-2,An Autoencoder model for dealing with missing ...,Um modelo Autoencoder para lidar com dados aus...,Arthur Dantas Mangussi,Ministério da Defesa (Brasil). Instituto Tecno...,São José dos Campos,NaN,NaN,NaN,Ana Carolina Lorena,...,Aprendizado computacional:Pré-processamento de...,2024-03-15,2024-09-14,NaN,Universidade de Coimbra (UC),Portugal,Pedro Henriques Abreu,Vários problemas podem deteriorar a qualidade ...,Various issues can deteriorate data quality in...,NaN
3,23/10398-3,HUB FUNDOS: plataforma computacional para a pa...,HUB FUNDOS: computational platform for the sta...,Aloisio Mota Rodrigues Junior,NaN,NaN,NaN,RTM Infraestrutura em Tecnologia da Informação...,São Paulo,Aloisio Mota Rodrigues Junior,...,Processamento de linguagem natural:Transformaç...,2024-03-01,2024-11-30,NaN,NaN,NaN,NaN,"Nos últimos dez anos, a indústria de Fundos no...","Over the last ten years, the Funds industry in...",NaN
4,23/06461-1,O uso de videogames no monitoramento e na manu...,The use of video games in monitoring and maint...,Fabio Ota,NaN,NaN,NaN,Eliza Massako Akiyama Ota,São Paulo,Fabio Ota,...,Inteligência artificial:Jogos de computador:Lo...,2024-03-01,2025-02-28,NaN,NaN,NaN,NaN,O envelhecimento populacional é um fenômeno ob...,Population aging is a phenomenon observed worl...,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7496,92/02151-8,Sistemas baseados em conhecimentos: metodologi...,NaN,Maria Carolina Monard,Universidade de São Paulo (USP). Instituto de ...,São Carlos,NaN,NaN,NaN,Maria Carolina Monard,...,Inteligência artificial:Sistemas baseados em c...,1992-02-01,1994-07-31,NaN,NaN,NaN,NaN,"O objetivo desta pesquisa é o estudo, desenvol...",NaN,NaN
7497,>>> Biblioteca Virtual da FAPESP (BV FAPESP) <<<,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7498,Dados gerados em: 15/Feb/24 12:10:22,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7499,URL: https://bv.fapesp.br/57069,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [4]:
df = df[df["Beneficiário"].notna() | df["Pesquisador Responsável"].notna()].reset_index(drop=True)

In [5]:
all_authors = {}
names = set(df["Beneficiário"].tolist()) | set(df["Pesquisador Responsável"].tolist())
for author in tqdm(names)modelomodelo:
    try:
        query = Authors().search_filter(display_name=author)
        all_authors[author] = []
        for record in chain(*query.paginate(per_page=200)):
            all_authors[author].append(record["id"])
    except:
        pass

100%|███████████████████████████| 4626/4626 [1:37:35<00:00,  1.27s/it]


In [9]:
import pickle as pkl
pkl.dump(all_authors, open("all_authors_fapesp.pkl", "wb"))

In [10]:
all_authors

{'Nivia Guilherme': ['https://openalex.org/A5029252769'],
 'Ariele Rossi Marques': [],
 'Rachel Miana Bezerra': [],
 'Matheus Vargas Volpon Berto': [],
 'Marcos Antônio Cavenaghi': ['https://openalex.org/A5038791485',
  'https://openalex.org/A5064307545'],
 'Marcos Vinicius Maciel da Silva': ['https://openalex.org/A5012853023'],
 'Washington Luís Figueiredo Soares': [],
 'Gustavo Toshio Yasunaka': [],
 'Marcelo Lima de Paiva': [],
 'Hércules de Araújo Feitosa': ['https://openalex.org/A5031259620',
  'https://openalex.org/A5068303742',
  'https://openalex.org/A5036181695'],
 'Wagner Farid Gattaz': ['https://openalex.org/A5070658874'],
 'Giovani Candido': [],
 'Paulo Eduardo Santos': ['https://openalex.org/A5057793121',
  'https://openalex.org/A5085163581',
  'https://openalex.org/A5059790674',
  'https://openalex.org/A5011236268',
  'https://openalex.org/A5029569554',
  'https://openalex.org/A5034430438',
  'https://openalex.org/A5087179162',
  'https://openalex.org/A5073464744',
  'htt

In [40]:
def get_metadata(dois: List[str], is_id=False):
    dois_queries = []
    max_size = 25
    
    if is_id == False:
        doi_prefix = "https://doi.org/"
        dois_cat = [doi_prefix + doi for doi in dois]
        while dois_cat: 
            chunk, dois_cat = dois_cat[:max_size], dois_cat[max_size:]
            query = "|".join(chunk)
            dois_queries.append(query)
    else:
        while dois: 
            chunk, dois = dois[:max_size], dois[max_size:]
            dois_queries.append(chunk)

    all_results = []
    for q in dois_queries:
        if is_id == False:
            results = Works().filter(doi=q).get()
        else:
            results = Works()[q]
        filtered_results = []
        for r in results:
            try:
                found_first_author = False
                for fa_idx in range(len(r["authorships"])):
                    if r["authorships"][fa_idx]['author_position'] == 'first':
                        found_first_author = True
                        break
                        
                sorted_keywords = [m["keyword"] for m in sorted(r["keywords"], key=lambda x: x['score'], reverse=True)]
                all_authors = [m["raw_author_name"] for m in r["authorships"]]
                sorted_authors_af = [m["raw_affiliation_string"] for m in r["authorships"]] 
                filtered_results.append({
                    "doi": str(r["doi"]),
                    "id": str(r["id"]),
                    "abstract": str(r["abstract"]),
                    "title": str(r["title"]),
                    "publication_date": str(r["publication_date"]),
                    "type": str(r["type"]),
                    "type_crossref": str(r["type_crossref"]),
                    "keywords": sorted_keywords,
                    "first_author": str(r["authorships"][fa_idx]['raw_author_name']) if found_first_author else "",
                    #"host_organization_name": str(r["primary_location"]["source"]["host_organization_name"]) if "source" in r["primary_location"] and ,
                    "all_authors": all_authors,
                    "all_authors_af": sorted_authors_af,
                    "language": r["language"],
                    "referenced_works": r["referenced_works"],
                    "referenced_works_count": r["referenced_works_count"],
                    "related_works": r["related_works"],
                    "concepts": [(c["display_name"], c["score"]) for c in r["concepts"]] if r["concepts"] else [],
                    "is_accepted": r["primary_location"]["is_accepted"],
                    "is_published": r["primary_location"]["is_published"],
                })
            except:
                pass
        all_results.extend(filtered_results)
    return all_results

In [26]:
#df_sample = df.sample(n=10_000, random_state=0)
unique_dois = df["citing_doi"].tolist() + df["cited_doi"].tolist()
unique_dois = list(set(unique_dois))

In [9]:
print(len(unique_dois))

668383


In [1]:
def parallel_openalex_queries(query_list, divs, is_id=False):
    N = len(query_list)
    piece_size = int(N / divs) + 1
    NUM_JOBS = 4
    all_data = []
    for div in range(divs):
        query_list_piece = query_list[div * piece_size:(div + 1) * piece_size]
        
        N_piece = len(query_list_piece)
        
        job_piece_size = int(N_piece / NUM_JOBS) + 1
        chunks = [query_list_piece[job_piece_size * i:job_piece_size * (i + 1)] for i in range(NUM_JOBS)]
        
        metadata = Parallel(n_jobs=NUM_JOBS)(delayed(get_metadata)(d, is_id=is_id) for d in tqdm(chunks, total=len(chunks)))
        metadata = list(itertools.chain.from_iterable(metadata))
        print(len(metadata))
        all_data.extend(metadata)
    return all_data

In [41]:
all_data = parallel_openalex_queries(unique_dois, 10, is_id=False)

100%|██████████████████████████████████| 4/4 [00:00<00:00, 133.42it/s]


66833


100%|█████████████████████████████████| 4/4 [00:00<00:00, 5845.72it/s]


66836


100%|█████████████████████████████████| 4/4 [00:00<00:00, 6150.01it/s]


...

66833


100%|█████████████████████████████████| 4/4 [00:00<00:00, 6538.28it/s]


66837


100%|█████████████████████████████████| 4/4 [00:00<00:00, 8890.95it/s]


66828


In [ ]:
extra_refs = []
for d in all_data:
    try:
        extra_refs.extend(eval(d["referenced_works"]))
    except:
        pass
extra_refs = list(set(extra_refs) - set([x["id"] for x in all_data]))

In [ ]:
extended_data = parallel_openalex_queries(extra_refs, 20, is_id=True)

In [42]:
df_metadata = pd.DataFrame(list(set(all_data + extended_data)))

In [47]:
len(df_metadata["doi"].unique()) #[df_metadata["abstract"] != ""]

668263

In [44]:
df_metadata.to_csv("metadata.csv")

In [45]:
df_metadata.to_pickle("metadata.pkl", compression={'method': 'gzip', 'compresslevel': 3, 'mtime': 1})